# Building a sample API call:

In [1]:
import requests
from xml.etree import ElementTree as ET

In [2]:
base_url = 'https://secure.shippingapis.com/ShippingAPI.dll?API=TrackV2&XML='
from config import userid, password

In [3]:
# Sample API call:

trackid = '9114902200852144849041'
# response = requests.get(base_url + \
#                         f'<TrackRequest USERID={userid} PASSWORD={password}>\
#                             <TrackID ID={trackid}></TrackID>\
#                                 </TrackRequest>')

response = requests.get(base_url + \
                        f'<TrackRequest USERID="{userid}" PASSWORD="{password}">\
                            <TrackID ID="{trackid}"></TrackID>\
                                </TrackRequest>')

# response = requests.get(base_url + \
#                         f'USERID={userid}&PASSWORD={password}\
#                             &ID={trackid}')
root = ET.fromstring(response.content)

In [4]:
str(response.content)

'b\'<?xml version="1.0" encoding="UTF-8"?>\\n<TrackResponse><TrackInfo ID="9114902200852144849041"><TrackSummary>Your item was delivered in or at the mailbox at 11:31 am on July 18, 2024 in WAXAHACHIE, TX 75165.</TrackSummary><TrackDetail>Arrived at USPS Regional Origin Facility, 07/17/2024, 3:10 pm, DALLAS TX DISTRIBUTION CENTER</TrackDetail><TrackDetail>Arrived at USPS Regional Origin Facility, 07/16/2024, 10:51 pm, COPPELL TX DISTRIBUTION CENTER</TrackDetail><TrackDetail>USPS picked up item, July 15, 2024, 5:38 pm, WACO, TX 76710</TrackDetail></TrackInfo></TrackResponse>\''

In [5]:
'Your item was delivered' in str(response.content)

True

In [6]:
print(root.tag)
for child in root:
    print(child.tag)
    for grandchild in child:
        print(grandchild.tag, grandchild.text)

TrackResponse
TrackInfo
TrackSummary Your item was delivered in or at the mailbox at 11:31 am on July 18, 2024 in WAXAHACHIE, TX 75165.
TrackDetail Arrived at USPS Regional Origin Facility, 07/17/2024, 3:10 pm, DALLAS TX DISTRIBUTION CENTER
TrackDetail Arrived at USPS Regional Origin Facility, 07/16/2024, 10:51 pm, COPPELL TX DISTRIBUTION CENTER
TrackDetail USPS picked up item, July 15, 2024, 5:38 pm, WACO, TX 76710


In [7]:
root.TrackSummary

AttributeError: 'xml.etree.ElementTree.Element' object has no attribute 'TrackSummary'

# To build the string:

In [ ]:
# import pandas as pd
# tracking_df = pd.read_csv('TrackingList.csv')

In [ ]:
# tracking_df.head()

In [ ]:
# url_b = f'<TrackRequest USERID="{userid3}" PASSWORD="{password3}">'
# i = 0
# for tnum in tracking_df['Tracking Nos.']:
#     while i <= 9:
#         url_b = url_b + (f'<TrackID ID="{tnum}"></TrackID>')
#         i += 1

# url_b = url_b +'</TrackingRequest>'
# url_b

# Another approach: an API call for each number:

This approach uses less code and is probably better, but will make many more api calls. The documentation does not have a limit on how many calls can be made.

In [10]:
# REITERATE:
# base_url = 'https://secure.shippingapis.com/ShippingAPI.dll?API=TrackV2&XML='
# from config import userid3, password3

import pandas as pd
tracking_df = pd.read_csv('TrackingList.csv',dtype={'Client Nos.':str})

print(tracking_df.tail())
print(tracking_df.dtypes)

              Tracking Nos. Client Nos.              Comments
113  9114902200852144849386    50836.02  1040 and Source Docs
114  9114902200852144849393    54159.02           Source Docs
115  9114902200852144849416    70250.02           Source Docs
116  9114902200852144849409    63909.02    Source Docs and EL
117  9114902200852144849423    14246.02  Form 1041; Attempt 2
Tracking Nos.    object
Client Nos.      object
Comments         object
dtype: object


In [ ]:
delivered_list = []
awaiting_pickup_list = []

for num in tracking_df['Tracking Nos.']:
    response = requests.get(base_url + \
                        f'<TrackRequest USERID="{userid}" PASSWORD="{password}">\
                            <TrackID ID="{num}"></TrackID>\
                                </TrackRequest>')
    if 'Your package was delivered' in response.content:
        delivered_list.append(num)
    elif 'Awaiting pickup' in response.content:
        awaiting_pickup_list.append(num)


    #   DoSomethingWith(response)
        # Pick out the 'TrackSummary' element from the tree.
        # If it includes 'Your item was delivered,' add it to a list
    # Check trackingpseudocode.md.

In [ ]:
# Create a delivered_list of imaginary packages to test the code
# below:
delivered_list = ['7019','7020','7021']

# Write a file with a list of delivered packages:

In [ ]:
print('The following ' + str(len(delivered_list)) + ' packages have been delivered:')
if delivered_list == []:
    print('None.')
else:
    [print(f'{id}') for id in delivered_list]

In [ ]:
import datetime

# Write a file to save the list of delivered packages:
savelist = 'x'
while savelist.lower() != 'y' and savelist.lower() != 'n':
    savelist = input('Would you like to save a list of them? (y/n) ')
    
    if savelist.lower() == 'y':
        # Write a file with date and time in the file name:
        rn = datetime.datetime.now()
        filename = 'Output\deliveredlist' + rn.strftime('%Y%m%d_%H%M') + '.txt'
        deliveredfile = open(filename, 'a')
        deliveredfile.write(
            'Packages delivered as of ' + rn.strftime('%d %B %Y, %H:%M') + ':\n'
        )
        for id in delivered_list:
            spot = tracking_df[tracking_df['Tracking Nos.'] == id].index[0]
            # NOTE: the above line is written with the assumption that exactly one row is returned. If the condition returns more than one index for a tracking number, the first will go into the varible.
            deliveredfile.write(f'\n{tracking_df.iloc[spot,0]}, {tracking_df.iloc[spot,1]}, {tracking_df.iloc[spot,2]}')
        deliveredfile.close()
        
        # With the file written, ask about removing the packages from the list:
        removedelivered = 'x'
        while removedelivered.lower() != 'y' and removedelivered.lower() != 'n':
            removedelivered = input('Would you like to remove them from the tracking list? (y/n) ')
            if removedelivered == 'y':
                for id in delivered_list:
                    tracking_df.drop(tracking_df[tracking_df['Tracking Nos.'] == id].index[0], inplace=True)
                    # NOTE: the above line is written with the assumption that exactly one row is returned. If the condition returns more than one index for a tracking number, only the first will be deleted.
                tracking_df.to_csv('Output\TrackingList' + rn.strftime('%Y%B%d_%H%M') + '.csv', index=False)
                tracking_df.to_csv('TrackingList.csv', index=False)
            elif removedelivered != 'y' and removedelivered != 'n':
                print('Please answer with the letter y for Yes or the letter n for No.')

    elif savelist != 'y' and savelist != 'n':
        print('Please answer with the letter y for Yes or the letter n for No.')

In [ ]:
# Playground cell
tracking_df[tracking_df['Comments'] == 'Imaginary'].index

In [ ]:
tracking_df.tail()
# tracking_df.drop(tracking_df[tracking_df['Tracking Nos.'] == id].index[0])

In [ ]:
tracking_df.to_csv('TrackingList.csv',index=False)

# Setting up the output file to include confirmation information:

In [ ]:
# From above:

# root = ET.fromstring(response.content)

# print(root.tag)
# for child in root:
#     print(child.tag)
#     for grandchild in child:
#         print(grandchild.tag, grandchild.text)

# Output:

# TrackResponse
# TrackInfo
# TrackSummary Your package is moving within the USPS network and is on track to be delivered to its final destination. It is currently in transit to the next facility.
# TrackDetail In Transit to Next Facility, 04/22/2024
# TrackDetail In Transit to Next Facility, 04/21/2024
# TrackDetail In Transit to Next Facility, 04/20/2024
# TrackDetail Arrived at USPS Regional Facility, April 18, 2024, 8:36 am, CHARLOTTE NC DISTRIBUTION CENTER
# TrackDetail In Transit to Next Facility, 04/17/2024
# TrackDetail Arrived at USPS Regional Facility, April 16, 2024, 12:34 am, COPPELL TX DISTRIBUTION CENTER


In [4]:
# Reminder: root = ET.fromstring(response.content)

from bs4 import BeautifulSoup

In [ ]:
soup = BeautifulSoup(response.content,'xml')

In [ ]:
soup.TrackInfo['ID']

In [ ]:
# So the line would be:
# deliveredfile.write('\n\n' + soup.TrackSummary.text)

# Adjusting the code to write the file as we go, so we can include summaries:

In [6]:
delivered_list = []
from datetime import datetime as dt

# Test on the tail of the dataframe (I've added two delivered packages to the end):
rn = dt.now()
filename = 'Output\deliveredlistSOUPED' + rn.strftime('%Y%m%d_%H%M') + '.txt'
deliveredfile = open(filename, 'a')
deliveredfile.write(
    'Packages delivered as of ' + rn.strftime('%d %B %Y, %H:%M') + ':\n'
)
for num in tracking_df.tail()['Tracking Nos.']:
    response = requests.get(base_url + \
                        f'<TrackRequest USERID="{userid}" PASSWORD="{password}">\
                            <TrackID ID="{num}"></TrackID>\
                                </TrackRequest>')
    soup = BeautifulSoup(response.content, 'xml')
    # Three different phrases might indicate that the package has reached the recipient:
    condition1 = 'Your item was delivered' in str(response.content)
    condition2 = 'Your item has been delivered' in str(response.content)
    condition3 = 'Your item was picked up' in str(response.content)

    if condition1 | condition2 | condition3:
        delivered_list.append(num)
        spot = tracking_df[tracking_df['Tracking Nos.'] == num].index[0]
        # NOTE: the above line is written with the assumption that exactly one row is returned. If the condition returns more than one index for a tracking number, the first will go into the varible.
        deliveredfile.write(f'\n{tracking_df.iloc[spot,0]}, {tracking_df.iloc[spot,1]}, {tracking_df.iloc[spot,2]}')
        deliveredfile.write('\n\n' + soup.TrackSummary.text + '\n')

deliveredfile.close()


# An adjustment:

Fixing the line in the writing cell so that it can handle multiple entries associated with one tracking number (i.e. multiple returns going to one person).

In [18]:
tracking_df = pd.read_csv('TestRun2\Output\TrackingList2024July29_1114.csv')
spots = tracking_df[tracking_df['Tracking Nos.'] == '9114902200852144849393'].index
print(spots)
spots[0]

Index([121], dtype='int64')


121

In [15]:
tracking_df.tail()

,Unnamed: 0,Tracking Nos.,Client Nos.,Comments
117,129,9589071052701128330960,11998.02,Multiple Returns
118,130,9589071052701128330960,11994.02,Multiple Returns
119,131,9589071052701128330960,17416.02,Multiple Returns
120,132,9114902200852144849386,50836.02,1040 and Source Docs
121,133,9114902200852144849393,54159.02,Source Docs


In [10]:
import pandas as pd
list = [11,12,13,13,13]
list2 = ['a','v','f','f','f']
list3 = ['a',2,3,4,5]
frame = pd.DataFrame([list,list2,list3],columns=['A','B','C','D','E'])
set(frame['A'])

{11, 'a'}